## Ilastik for 10/31/16 Week
From last week, I was able to generate 3D TIFF slices and image classifiers on Fear199 downsampled data.  However, my problems were that:  
1)  The TIFF slices were odd, cigar-shaped tubes.  
2)  I was unable to generate a significant classifier using the existing data because of the weird image layout.  

What I did this week was:  
1)  Figure out why my original data was the odd cigar-shaped data.    
2)  Correctly generate a subset of TIFF slices for Fear199.   
3)  Generate a pixel-based object classifier.  

### Task 1: Why was my original data cigar-shaped?  
When downloading the image from ndreg, there were two different approaches to generating the numpy array.  I've shown both below:

In [ ]:
## Script used to download nii run on Docker
from ndreg import *
import matplotlib
import ndio.remote.neurodata as neurodata
import nibabel as nb
inToken = "Fear199"
nd = neurodata()
print(nd.get_metadata(inToken)['dataset']['voxelres'].keys())
inImg = imgDownload(inToken, resolution=5)
imgWrite(inImg, "./Fear199.nii")

In [ ]:
## Method 1:
import os
import numpy as np
from PIL import Image
import nibabel as nib
import scipy.misc
TokenName = 'Fear199.nii'
img = nib.load(TokenName)

## Convert into np array (or memmap in this case)
data = img.get_data()
print data.shape
print type(data)

In [ ]:
## Method 2:
rawData = sitk.GetArrayFromImage(inImg)  ## convert to simpleITK image to normal numpy ndarray
print type(rawData)

In a nutshell, method 1 generates an array with shape (x, y, z) -- specifically, (540, 717, 1358).  The method 2 generates a numpy array with shape (z, y, x) -- specifically, (1358, 717, 540).  Since we want the first column to be z slices, the original method was granting me x-slices (hence the cigar-tube dimensions).

In order to interconvert, we can either just use the rawData approach after directly calling from ndstore, or we can take our numpy array after loading from nibabel and use numpy's swapaxes method to just swap two of the dimensions (shown below).

In [ ]:
## if we have (i, j, k), we want (k, j, i)  (converts nibabel format to sitk format)
new_im = newer_img.swapaxes(0,2) # just swap i and k

### Task 2: Generating raw TIFF slices.
Now that I have appropiate coordinates, I generated a subset of TIFF slices to run the training module for the image classifier.  Using the script here:

In [ ]:
plane = 0;
for plane in (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 100, 101, 102, 103, 104):
    output = np.asarray(rawData[plane])
    ## Save as TIFF for Ilastik
    scipy.misc.toimage(output).save('RAWoutfile' + TokenName + 'ITK' + str(plane) + '.tiff')

As shown above, I generated data for the first 13 planes, and then some subset of planes from z = 100 to z = 104.  I then trained the classifier on the 0 through 12 data, and used the 100 to 104 slices to validate my results.  Below I've included images of the raw and histogram equalized images at one specific slice to just show the raw images I was working with.

**Raw Fear 199 at Z = 100:**
![rawFear199Slice100](images/oct31/RAWoutfileFear199.niiITK100.png)

**Histogram Equalized Fear 199 at Z = 100:**
![histeqFear199Slice100](images/oct31/HISTEQoutfileFear199ITK100.png)

In order to generate the histogram equalized TIFF slices, I wrote a Jupyter notebook here:
https://neurodatadesign.github.io/seelviz/tony/html/generate+histEQ.html

### Task 3:  Generate a pixel-based object-classifier for both the raw and the histogram equalized data.
From there, I generated a pixel-based object-classifier for both the raw and the histogram equalized data.  Shown first are images demonstrating me generating the classifier for the raw data.  Then I show the images of me generating the classifier for the histogram equalized data.

First, the raw data classifier.  I load in planes 0-12 TIFF slices and generate a classifier by training it to select for borders, background, and individual bright points:

**Loading Raw Training Data**
![loadingrawFear199Training](images/oct31/loadingrawfear199.png)

**Selecting Borders in Raw Training Data**
![trainingbordersFear199](images/oct31/trainingonbordersrawfear199.png)

**Selecting Bright Points in Raw Training Data**
![trainingbrightpointsFear199](images/oct31/trainingonbrightpointsrawfear199.png)

**Closeup of Feature Selection in Raw Training Data**
![closeupfeatureselectionrawfear199](images/oct31/closeupfeatureselectionrawfear199.png)

**Object Output for Raw Training Data**
![exampleobjectsrawfear199](images/oct31/exampleobjectsrawfear199.png)

In order to run Ilastik headlessly on 3D classified data, use globstring syntax to tell ilastik which images to combine for each volume.  EG:

For me to get it to work, I made a folder called "runtime" where each file was named WITHOUT the underscores.  Make sure to add the * to get it to work, also.

Repeating the process for the histogram equalized brains, we get:

### Future goals:

Eventually, we want our methodology to do something similar to this: http://ilastik.org/documentation/counting/counting.  We want to be able to use our pixel-based classifier to try to find the number of neurons in a given 3D image.

However, much to my chagrin, this methodology currently does NOT support 3D data -- it only supports 2D data.  I can apply the technique to 2D 
    